# @chain 데코레이터와 스트리밍 결합하기

이 노트북에서는 **@chain 데코레이터**와 **스트리밍(yield)**을 함께 사용하여 토큰 단위로 응답을 전달하는 방법을 알아봅니다.

## 스트리밍이 필요한 이유

LLM 응답은 생성에 시간이 걸립니다. 스트리밍을 사용하면:

1. **체감 대기 시간 감소**: 첫 토큰이 생성되면 바로 표시
2. **사용자 경험 향상**: ChatGPT처럼 글자가 하나씩 나타나는 효과
3. **실시간 피드백**: 응답이 진행 중임을 사용자가 인지

## @chain + 스트리밍 원리

```python
@chain
def my_chain(values):
    for token in model.stream(prompt):
        yield token  # return 대신 yield 사용!
```

**핵심:**
- `return` 대신 **`yield`** 사용 → 제너레이터 함수
- `model.stream()`의 각 토큰을 **그대로 전달**
- `chatbot.stream()`으로 호출하면 토큰 단위로 수신

## 실행 흐름

```

   입력값        ──▶    Prompt      ──▶  model.stream   ──▶     yield     
  {question}            생성                 (토큰)              (토큰)     
                                                                    
                                                      ┌───────────────┘
                                                      ▼
                                          chatbot.stream() 호출 측에서
                                          토큰을 하나씩 수신
```

---

# 1. Ollama 설치 및 서버 실행

In [1]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 0s (2,032 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current u

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

In [2]:
!ollama pull llama3.2
!pip install -q langchain-ollama

# 3. 구성 요소 준비

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

# Model
model = ChatOllama(model='llama3.2')

# Prompt Template
template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}'),
])

# 4. @chain + yield로 스트리밍 체인 구성

**코드 설명:**

### yield 키워드의 역할
```python
@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in model.stream(prompt):
        yield token  # 각 토큰을 즉시 반환
```

- `yield`는 함수를 **제너레이터**로 만듦
- 각 토큰이 생성될 때마다 **즉시** 호출 측으로 전달
- 전체 응답을 기다리지 않고 **실시간 처리** 가능

### return vs yield 차이

| 키워드 | 동작 | 결과 |
|--------|------|------|
| `return` | 모든 토큰 수집 후 반환 | 전체 응답 한 번에 |
| `yield` | 각 토큰 즉시 반환 | 토큰 단위 스트리밍 |

In [4]:
from langchain_core.runnables import chain

# 스트리밍을 지원하는 체인
@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in model.stream(prompt):
        yield token  # return 대신 yield!

# 5. 스트리밍 실행

`chatbot.stream()`을 호출하면 토큰이 하나씩 전달됩니다.

In [5]:
# 스트리밍 실행 - 토큰 단위 출력
print("=== 스트리밍 출력 ===")

for part in chatbot.stream({'question': '거대 언어 모델은 어디서 제공하나요?'}):
    print(part.content, end='', flush=True)

print()  # 줄바꿈

=== 스트리밍 출력 ===
Large language models, 특히 BERT, RoBERTa, XLNet 및 others가 포함된 언어 모델은 일반적으로 대량 데이터에 entren됩니다. 이러한 modelos는 여러来源에서 데이터를 받을 수 있습니다:

1. **공식 데이터**: 언어 모델의 개발자들은 souvent 자신의 dữ liệu를 사용합니다. 예를 들어, BERT는 유럽 연합의 유럽언어 데이터베이스 (EUCLID)와 Common Crawl에서 제공하는 웹 크롤링 데이터를ใช하여 훈련되었습니다.
2. **공공 데이터**: 공공 데이터베이스나 데이터베이스에서 데이터를 얻을 수 있습니다. 예를 들어, 유사한 데이터가 포함된 US National Archives & Records Administration (NARA) 데이터베이스는 다양한 언어 모델의 훈련에 사용됩니다.
3. **인터넷 크롤링**: 대량 인터넷 크롤링은 일반적이며 많은 언어 모델은 이 방법을 사용하여 데이터를 수집합니다. 이러한 데이터는 웹 trang, 포스트 및 다른 온라인 콘텐츠에서 얻어질 수 있습니다.
4. **사용자 데이터**: 일부 언어 môdel은 사용자가 수집하고 제공하는 데이터를 사용합니다. 예를 들어, 데이터의 특정한 주제 또는 용어에 대한 더 나은 이해를 위해 사용자에 의한 데이터가 제공될 수 있습니다.
5. **가상 데이터**: 가상 데이터 생성 도구와 같은 기술을 사용하여 가상 데이터를สร้าง하는 방법도 있습니다. 이러한 데이터는 실제-world dữ liệu의 특징과 유사합니다.

다음은 일부 lớn한 언어 모델이 whence coming이란 그들의 training 데이터에 대한 예시입니다:

* **BERT**: 유럽 연합의 EUCLID, Common Crawl
* **RoBERTa**: WebScraper (Common Crawl), Wikipedia (Wikidata)
* **XLNet**: BookCorpus, Wikipedia ( Wikidata)



# 6. 각 토큰(chunk) 상세 확인

스트리밍에서 각 `part`가 어떤 구조인지 확인해봅니다.

In [6]:
# 각 chunk 상세 확인
print("=== 각 chunk 상세 ===")

chunks = []
for i, part in enumerate(chatbot.stream({'question': '안녕!'})):
    chunks.append(part.content)
    if i < 10:  # 처음 10개만 출력
        print(f"chunk[{i}]: type={type(part).__name__}, content='{part.content}'")

print(f"...\n총 {len(chunks)}개 chunk")
print(f"\n전체 응답: {''.join(chunks)}")

=== 각 chunk 상세 ===
chunk[0]: type=AIMessageChunk, content='hello'
chunk[1]: type=AIMessageChunk, content='!'
chunk[2]: type=AIMessageChunk, content=''
chunk[3]: type=AIMessageChunk, content=''
...
총 4개 chunk

전체 응답: hello!


# 7. invoke vs stream 비교

같은 체인에서 `invoke()`와 `stream()` 차이를 비교합니다.

In [7]:
import time

question = {'question': 'Python이 뭔가요? 간단히 설명해주세요.'}

# invoke() - 전체 응답 대기
print("=== invoke() ===")
start = time.time()

# yield를 사용하는 함수는 invoke()시 제너레이터를 반환
# 전체 결과를 얻으려면 list()로 변환하거나 순회해야 함
result = list(chatbot.invoke(question))
full_response = ''.join([r.content for r in result])

print(f"전체 응답 시간: {time.time() - start:.2f}초")
print(f"응답: {full_response[:100]}...")

print("\n" + "="*50 + "\n")

# stream() - 토큰 단위 수신
print("=== stream() ===")
start = time.time()
first_token_time = None

response_parts = []
for part in chatbot.stream(question):
    if first_token_time is None:
        first_token_time = time.time() - start
    response_parts.append(part.content)

print(f"첫 토큰 수신: {first_token_time:.2f}초")
print(f"전체 응답 시간: {time.time() - start:.2f}초")
print(f"응답: {''.join(response_parts)[:100]}...")

=== invoke() ===


AttributeError: 'tuple' object has no attribute 'content'

---

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI)
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI(model='gpt-3.5-turbo')

# 변경 (Ollama)
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## 스트리밍 패턴 비교

### 1. 직접 model.stream() 사용 (10번 노트북)
```python
for token in model.stream('질문'):
    print(token.content, end='')
```

### 2. LCEL 체인에서 stream() (권장)
```python
chain = template | model
for token in chain.stream({'question': '질문'}):
    print(token.content, end='')
```

### 3. @chain + yield (이 노트북)
```python
@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in model.stream(prompt):
        yield token

for token in chatbot.stream({'question': '질문'}):
    print(token.content, end='')
```

## 언제 @chain + yield를 사용할까?

| 상황 | 추천 방식 |
|------|----------|
| 단순 체인 스트리밍 | LCEL `chain.stream()` |
| 스트리밍 중 **로깅** 필요 | @chain + yield |
| 스트리밍 중 **토큰 변환** 필요 | @chain + yield |
| 스트리밍 중 **조건부 처리** 필요 | @chain + yield |

## 스트리밍 중 토큰 변환 예시

```python
@chain
def uppercase_chatbot(values):
    prompt = template.invoke(values)
    for token in model.stream(prompt):
        # 토큰 내용을 대문자로 변환
        token.content = token.content.upper()
        yield token
```